In [ ]:
!python cornell_demo.py --replay_flor


Test set: Average loss: 0.0414, Accuracy: 9858/10000 (99%)

/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)

Test set: Average loss: 0.0421, Accuracy: 9870/10000 (99%)


Test set: Average loss: 0.0326, Accuracy: 9887/10000 (99%)


Test set: Average loss: 0.0304, Accuracy: 9903/10000 (99%)


Test set: Average loss: 0.0310, Accuracy: 9900/10000 (99%)



In [ ]:
!python cornell_demo.py --flor cornell-demo

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.314478
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.712248
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.088857
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.491460
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.174030
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.529356
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.308861
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.305906
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.295615
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.129737
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.354210
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.098806
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.137030
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.237691
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.178311
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.263660
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.165038
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.292757
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.230463
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.280855
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.055802
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.078345
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.009530
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.015425
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.060847
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.023271
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.117742
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.177705
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.095329
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.089770
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.053652
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.004115
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.047938
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.108757
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.014441
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.066097
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.003798
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.021224
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.017956
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.008585


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.005703
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.011609
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.068438
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.027901
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.109267
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.005691
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.002496
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.007291
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.018310
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.137267
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.005422
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.046808
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.061263
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.103052
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.008893
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.020097
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.008976
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.024209
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.048298
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.011949


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.001926
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.008438
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.028228
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.011836
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.036422
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.005552
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.012354
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.029161
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.077999
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.040609
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.000579
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.067782
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.052248
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.004323
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.011929
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.012925
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.086260
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.052898
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.046799
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.011687


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.066545
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.037294
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.179004
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.043274

Test set: Average loss: 0.0262, Accuracy: 9908/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.029354
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.015986
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.006792
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.015681
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.004407
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.002965
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.005579
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.009133
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.045253
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.026958
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.019947
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.004441
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.059906
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.011249
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.062097
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.020684
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.135046
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.014591
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.036637
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.056376
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.003284
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.015561
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.006437
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.054964
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.002803
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.003798
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.027097
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.001520
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.029839
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.018954
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.004864
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.031684
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.019211
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.017948
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.023855
